In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, Trainer, TrainingArguments, DataCollatorForLanguageModeling

from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

import pandas as pd

D:\.conda_env\yothalia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin D:\.conda_env\yothalia\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


## Load Model and Adapter

In [2]:
model = AutoModelForCausalLM.from_pretrained("D:/Projects/project_yothalia/yothalia/server/model_weights/internlm/internlm-chat-7b-finetune-int8", 
                                                load_in_4bit=True,
                                                #peft_config=config,
                                                trust_remote_code=True)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("D:/Projects/project_yothalia/yothalia/server/model_weights/internlm/internlm-chat-7b-finetune-int8",
                                            trust_remote_code=True)

In [7]:
peft_model_id = "D:/Projects/project_yothalia/yothalia/server/model_weights/internlm/internlm-chat-7b-finetune-lora"
config = PeftConfig.from_pretrained(peft_model_id)
#model = PeftModel.from_pretrained(model, peft_model_id)
model.add_adapter(config)

ValueError: Adapter with name default already exists. Please use a different name.

In [8]:
model.print_trainable_parameters()

AttributeError: 'InternLMForCausalLM' object has no attribute 'print_trainable_parameters'

## Set Trainer

#### Load dataset

In [5]:
from torch.utils.data import DataLoader
import pandas as pd

In [21]:
df = pd.read_csv('../train_sample/csv/train.csv',index_col=0)

In [23]:
df = df.map(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512))

In [24]:
df.head()

,train
0,"[input_ids, attention_mask]"
1,"[input_ids, attention_mask]"
2,"[input_ids, attention_mask]"
3,"[input_ids, attention_mask]"
4,"[input_ids, attention_mask]"


In [39]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=3,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  #max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=4,

  # Directory to save model checkpoints
  output_dir='./ckp',

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=4, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adamw_torch",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  greater_is_better=False
)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [40]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=df["train"][0:200],
    eval_dataset=df["train"][-20:-1],
    data_collator=data_collator,
)

In [41]:
trainer.train()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn